In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from datasets import load_dataset
from huggingface_hub import login, logout

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mozilla-data-prep/temp_output.txt
/kaggle/input/mozilla-data-prep/temp_input.txt
/kaggle/input/mozilla-data-prep/auto_ipa_annotations.csv
/kaggle/input/mozilla-data-prep/__results__.html
/kaggle/input/mozilla-data-prep/__huggingface_repos__.json
/kaggle/input/mozilla-data-prep/__notebook__.ipynb
/kaggle/input/mozilla-data-prep/install_mfa.sh
/kaggle/input/mozilla-data-prep/__output__.json
/kaggle/input/mozilla-data-prep/custom.css


In [2]:
import json

In [3]:
# remember to add user secret if you haven't already
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")

login(secret_value_0)

In [4]:
"""
example format for annotations and audio from https://labelstud.io/guide/tasks#Basic-Label-Studio-JSON-format
To place multiple tasks in one JSON file, use this JSON format example
This example contains multiple text classification tasks with no annotations or predictions.

The “data” parameter must contain the “my_text” field defined in the text labeling config and can optionally include other fields. The “id” parameter is not required.
[
   {
      "id":1,
      "data":{
         "my_text":"Opossums like to be aloft in trees."
      }
   },
   {
      "id":2,
      "data":{
         "my_text":"Opossums are opportunistic."
      }
   },
   {
      "id":3,
      "data":{
         "my_text":"Opossums like to forage for food."
      }
   }
]
"""

'\nexample format for annotations and audio from https://labelstud.io/guide/tasks#Basic-Label-Studio-JSON-format\nTo place multiple tasks in one JSON file, use this JSON format example\nThis example contains multiple text classification tasks with no annotations or predictions.\n\nThe “data” parameter must contain the “my_text” field defined in the text labeling config and can optionally include other fields. The “id” parameter is not required.\n[\n   {\n      "id":1,\n      "data":{\n         "my_text":"Opossums like to be aloft in trees."\n      }\n   },\n   {\n      "id":2,\n      "data":{\n         "my_text":"Opossums are opportunistic."\n      }\n   },\n   {\n      "id":3,\n      "data":{\n         "my_text":"Opossums like to forage for food."\n      }\n   }\n]\n'

label studio format for preannotated data: https://labelstud.io/guide/predictions.html

In [5]:
# generated annotations
annotated_df = pd.read_csv("/kaggle/input/mozilla-data-prep/auto_ipa_annotations.csv")

In [6]:
annotated_df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,client_id,path,audio,sentence,up_votes,down_votes,age,gender,accent,locale,segment,variant,ipa_auto,ipa_manual
0,0391ea7402fa4f593b7b9a74acd72a4e1c785e0bec9bae...,/root/.cache/huggingface/datasets/downloads/ex...,"{'bytes': None, 'path': '/root/.cache/huggingf...","A Pheig, inis scéal dúinn.",2,0,NaN,NaN,Gaeilge Chonnacht,ga-IE,NaN,NaN,ə fʲ e ɟ ˈ i nʲ ə ʃ ˈ ʃ c eː l̻ˠ ˈ d̪ˠ uː n̻ʲ,NaN
1,04f4b463df3fdc27ccb4d4657b2502796ef88ad0df66e3...,/root/.cache/huggingface/datasets/downloads/ex...,"{'bytes': None, 'path': '/root/.cache/huggingf...",Tá peann agamsa agus tá peann agatsa agus is l...,2,0,NaN,NaN,NaN,ga-IE,NaN,NaN,ˈ t̪ˠ aː ˈ pʲ a n̻ˠ ˈ ə ɡ ə mˠ sˠ ə ˈ ə ɡ ə sˠ...,NaN
2,18653adfb4042d6b8dc10929dc8f100b5a7f9d7b201f9d...,/root/.cache/huggingface/datasets/downloads/ex...,"{'bytes': None, 'path': '/root/.cache/huggingf...",Chuir Siobhán fios ar an otharchar.,2,0,teens,male,NaN,ga-IE,NaN,NaN,ˈ x u ɾʲ ʃ i vˠ a n̻ˠ ˈ fʲ i sˠ ˈ e ɾʲ ˈ ə n̻ˠ...,NaN
3,1f2d16e480d71c6f012d75aeca65e295e982ee88de51bf...,/root/.cache/huggingface/datasets/downloads/ex...,"{'bytes': None, 'path': '/root/.cache/huggingf...",Ag an bhfuinneog,2,0,NaN,NaN,NaN,ga-IE,NaN,NaN,ˈ e ɟ ˈ ə n̻ˠ vˠ i n̻ʲ o ɡ,NaN
4,2ddad8280f5eb089ac32b5c890d9427f0cab893ea91ebc...,/root/.cache/huggingface/datasets/downloads/ex...,"{'bytes': None, 'path': '/root/.cache/huggingf...",Pá seachtaine atá ag dul dom.,2,0,NaN,NaN,NaN,ga-IE,NaN,NaN,ˈ pˠ aː j ˈ ʃ a ɾˠ t̪ˠ ə nʲ ə ə ˈ t̪ˠ aː ˈ e ɟ...,NaN


In [7]:
# dataset with audio from hub
cv_13 = load_dataset("mozilla-foundation/common_voice_13_0", "ga-IE",split="test", trust_remote_code=True)

README.md:   0%|          | 0.00/14.7k [00:00<?, ?B/s]

common_voice_13_0.py:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.65k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

n_shards.json:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

...


Reading metadata...: 511it [00:00, 102963.55it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 4226it [00:00, 83538.56it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 882it [00:00, 103453.01it/s]


In [8]:
# make audio directory
base_dir = os.path.dirname("/kaggle/working/")

audio_dir = os.path.join(base_dir, "audio")
os.makedirs(audio_dir, exist_ok=True)


In [9]:
import os
import shutil
import json
import subprocess

In [10]:
json_list = []

for i, sample in enumerate(cv_13):
    sentence = annotated_df.iloc[i]["ipa_auto"]
    original_audio_path = sample["audio"]["path"]
    gsutil_URI = "gs://label-studio-ulster-thesis"

    # Create zero-padded filename
    file_id = f"{i+1:03d}"
    output_filename = f"{file_id}.wav"
    output_path = os.path.join(audio_dir, output_filename)

    # Convert to WAV and save using ffmpeg (commented out for only fetching json list)
    #subprocess.run([
    #    "ffmpeg", "-y", "-i", original_audio_path, "-ar", "16000", "-ac", "1", output_path
    #], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Add entry to JSON (full audio path adding gsutilURI)
    json_list.append({
      "id": f"{file_id}",
      "data":{
              "audio": f"{gsutil_URI}/audio/{output_filename}",
              "transcription": sentence
             }
    })

In [11]:
json_list[:5]

[{'id': '001',
  'data': {'audio': 'gs://label-studio-ulster-thesis/audio/001.wav',
   'transcription': 'ə fʲ e ɟ ˈ i nʲ ə ʃ ˈ ʃ c eː l̻ˠ ˈ d̪ˠ uː n̻ʲ'}},
 {'id': '002',
  'data': {'audio': 'gs://label-studio-ulster-thesis/audio/002.wav',
   'transcription': 'ˈ t̪ˠ aː ˈ pʲ a n̻ˠ ˈ ə ɡ ə mˠ sˠ ə ˈ ə ɡ ə sˠ ˈ t̪ˠ aː ˈ pʲ a n̻ˠ ˈ ə ɡ ə t̪ˠ sˠ ə ˈ ə ɡ ə sˠ ˈ i sˠ ˈ lʲ i n̻ʲ ˈ h eː nʲ ˈ ia d̪ˠ'}},
 {'id': '003',
  'data': {'audio': 'gs://label-studio-ulster-thesis/audio/003.wav',
   'transcription': 'ˈ x u ɾʲ ʃ i vˠ a n̻ˠ ˈ fʲ i sˠ ˈ e ɾʲ ˈ ə n̻ˠ ʌ hˠ a ɾˠ a x ɾˠ'}},
 {'id': '004',
  'data': {'audio': 'gs://label-studio-ulster-thesis/audio/004.wav',
   'transcription': 'ˈ e ɟ ˈ ə n̻ˠ vˠ i n̻ʲ o ɡ'}},
 {'id': '005',
  'data': {'audio': 'gs://label-studio-ulster-thesis/audio/005.wav',
   'transcription': 'ˈ pˠ aː j ˈ ʃ a ɾˠ t̪ˠ ə nʲ ə ə ˈ t̪ˠ aː ˈ e ɟ ˈ d̪ˠ u l̻ˠ ˈ d̪ˠ uː'}}]

In [12]:
# save the list in json format
with open(os.path.join(base_dir, "annotations.json"), "w", encoding="utf-8") as f:
    json.dump(json_list, f, ensure_ascii=False, indent=2)
